<a href="https://colab.research.google.com/github/Jorya777/Jorya777/blob/main/final_assignment_CF_evaluation_dashboard.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [15]:
!pip install streamlit plotly pandas numpy
!wget -q https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64.deb
!dpkg -i cloudflared-linux-amd64.deb >/dev/null 2>&1 || true


In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [10]:
%%writefile /content/drive/MyDrive/app.py
import streamlit as st
import pandas as pd
import plotly.express as px
import numpy as np

# ---------------------------------------------------
# 📘 基础配置
# ---------------------------------------------------
st.set_page_config(page_title="UNSDCF Evaluation Dashboard", layout="wide")

st.title("🌍 United Nations Sustainable Development Cooperation Framework Evaluation Dashboard")
st.markdown("""
This dashboard visualizes **evaluation expenditures** and **UNCT performance**
under the OECD-DAC evaluation criteria across selected countries.
""")

# ---------------------------------------------------
# 📊 加载真实数据
# ---------------------------------------------------
file_path = "/content/drive/MyDrive/2021-2023 evaluation expenditures analysis .xlsx"

df_spend = pd.read_excel(file_path)
df_spend.rename(columns={
    "Countries": "Country",
    "Evaluation expenditure($)": "Evaluation Spending (MUSD)",
    "Program Expenditure": "Total Spending (MUSD)",
    "The proportion of Evaluation Expenditure to Program Expenditure": "Eval Ratio (%)"
}, inplace=True)

# 清理潜在空格和异常值
df_spend["Country"] = df_spend["Country"].astype(str).str.strip()
df_spend["Eval Ratio (%)"] = pd.to_numeric(df_spend["Eval Ratio (%)"], errors="coerce")
df_spend.dropna(subset=["Eval Ratio (%)"], inplace=True)

countries = df_spend["Country"].unique().tolist()

# ---------------------------------------------------
# 🌍 全球填色地图
# ---------------------------------------------------
st.subheader("🌍 Global Distribution of Evaluation Expenditures (2021–2023)")

fig_map = px.choropleth(
    df_spend,
    locations="Country",
    locationmode="country names",
    color="Eval Ratio (%)",
    hover_name="Country",
    hover_data={
        "Evaluation Spending (MUSD)": ":,.2f",
        "Total Spending (MUSD)": ":,.2f",
        "Eval Ratio (%)": ":.2f"
    },
    color_continuous_scale="Blues",
    title="Evaluation Spending Ratio by Country",
    projection="natural earth"
)
fig_map.update_layout(geo=dict(showframe=False, showcoastlines=True))
st.plotly_chart(fig_map, use_container_width=True)

# ---------------------------------------------------
# 💰 散点图：Evaluation vs Programme Spending
# ---------------------------------------------------
st.subheader("💰 Evaluation vs Programme Expenditure")

fig_scatter = px.scatter(
    df_spend,
    x="Total Spending (MUSD)",
    y="Eval Ratio (%)",
    size="Evaluation Spending (MUSD)",
    color="Region" if "Region" in df_spend.columns else "Country",
    hover_name="Country",
    title="Evaluation Ratio vs Programme Expenditure",
    size_max=40
)
st.plotly_chart(fig_scatter, use_container_width=True)

# ---------------------------------------------------
# 📈 雷达图（示例数据）
# ---------------------------------------------------
st.subheader("📈 UNCT Performance under the UNSDCF (by OECD-DAC Criteria)")

CRITERIA = ['relevance','coherence','effectiveness','efficiency','orientation towards impact','sustainability']
# 用随机数模拟各国评分（如果你之后有真实打分表可以替换这里）
np.random.seed(42)
df_scores = pd.DataFrame({
    "Country": np.repeat(countries, len(CRITERIA)),
    "Criterion": CRITERIA * len(countries),
    "Score": np.random.randint(-3, 5, len(CRITERIA)*len(countries))
})

# Sidebar
country = st.sidebar.selectbox("Select Country", countries)
radar_df = df_scores[df_scores["Country"]==country]

fig_radar = px.line_polar(
    radar_df, r="Score", theta="Criterion", line_close=True,
    title=f"{country} – OECD-DAC Evaluation Balance",
    range_r=[-5,5]
)
st.plotly_chart(fig_radar, use_container_width=True)

st.markdown(
    "<p style='font-size:14px; color:gray;'><em>Note: A common UN benchmark is to allocate approximately 1% of programme resources to evaluation activities.</em></p>",
    unsafe_allow_html=True
)

# ---------------------------------------------------
# 📊 详细表格
# ---------------------------------------------------
st.subheader("📋 Detailed Evaluation Data")
st.dataframe(df_spend)

st.markdown("---")
st.markdown("© United Nations DCO – Data visualization for learning purposes ")


Overwriting /content/drive/MyDrive/app.py


In [14]:
import pandas as pd

# 1️⃣ 读取 Excel 文件
df_spend = pd.read_excel("/content/drive/MyDrive/2021-2023 evaluation expenditures analysis .xlsx")

# 2️⃣ 看看列名
print(df_spend.columns)

# 3️⃣ 看前几行数据
print(df_spend.head())


Index(['Countries', 'Region', 'Evaluation year ', ' Evaluation expenditure($)',
       'Program  Expenditure',
       'The proportion of Evaluation Expenditure to Program Expenditure'],
      dtype='object')
             Countries  Region  Evaluation year    Evaluation expenditure($)  \
0                Gabon  Africa              2021                       38500   
1               Chile      LAC              2021                       23610   
2  Republic of Moldova     ECA              2021                       61250   
3               Benin   Africa              2022                       87000   
4           Cape Verde  Africa              2021                       72350   

   Program  Expenditure  \
0              18233910   
1              31219623   
2              40731667   
3              46744773   
4              70865394   

   The proportion of Evaluation Expenditure to Program Expenditure  
0                                           0.002111                
1         

In [13]:
print(df_spend.columns)
print(df_spend.head())


NameError: name 'df_spend' is not defined

In [11]:
import subprocess, time, re, sys

PORT = "8501"
# 启动 Streamlit 后台服务
streamlit = subprocess.Popen(
    ["streamlit", "run", "app.py", "--server.port", PORT, "--server.headless", "true"],
    stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True
)
time.sleep(2)

# 启动 Cloudflare Tunnel
tunnel = subprocess.Popen(
    ["cloudflared", "tunnel", "--url", f"http://localhost:{PORT}", "--no-autoupdate"],
    stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True
)

print("Starting Cloudflare Tunnel … (keep this cell running)")
for line in tunnel.stdout:
    sys.stdout.write(line)
    sys.stdout.flush()
    m = re.search(r"https://[a-z0-9-]+\.trycloudflare\.com", line)
    if m:
        print("\n🚀 Streamlit app URL:", m.group(0))
        print("No password required. Keep this cell running while you use the app.")
        break


Starting Cloudflare Tunnel … (keep this cell running)
2025-10-23T20:58:48Z INF Thank you for trying Cloudflare Tunnel. Doing so, without a Cloudflare account, is a quick way to experiment and try it out. However, be aware that these account-less Tunnels have no uptime guarantee, are subject to the Cloudflare Online Services Terms of Use (https://www.cloudflare.com/website-terms/), and Cloudflare reserves the right to investigate your use of Tunnels for violations of such terms. If you intend to use Tunnels in production you should use a pre-created named tunnel by following: https://developers.cloudflare.com/cloudflare-one/connections/connect-apps
2025-10-23T20:58:48Z INF Requesting new quick Tunnel on trycloudflare.com...
2025-10-23T20:58:52Z INF +--------------------------------------------------------------------------------------------+
2025-10-23T20:58:52Z INF |  Your quick Tunnel has been created! Visit it at (it may take some time to be reachable):  |
2025-10-23T20:58:52Z INF | 